In [ ]:
import librosa.display, os, gc
import numpy as np
import matplotlib.pyplot as plt
import scaper, os
from keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D, AveragePooling2D

In [ ]:
import os
from keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
import random
from keras.callbacks import History

samples = []
labels = []
images_folder = "images/"
image_list = os.listdir(images_folder)
random.seed(10)
random.shuffle(image_list)

for image in image_list:
    samples.append(img_to_array(load_img(images_folder + image, target_size = (200, 200)))) # Change Target Size accordingly to the resolution.
#     print(image)
    if "normal" in image:
        labels.append((0))
    else:
        labels.append((1))
        
samples = np.array(samples)
labels = np.array(labels)
eval_x, eval_y = samples[-100:], labels[-100:] # 1000 images each for Normal and Soundscape, 100 in evaluation.
print(samples.shape, labels.shape)

x_train, x_test, y_train, y_test = train_test_split(samples[:-100], labels[:-100], test_size = 0.2, random_state = 31)

In [ ]:
def inception_module(x,
                     filters_1x1,
                     filters_3x3_reduce,
                     filters_3x3,
                     filters_5x5_reduce,
                     filters_5x5,
                     filters_pool_proj,
                     name=None):
    
    conv_1x1 = Conv2D(filters_1x1, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    
    conv_3x3 = Conv2D(filters_3x3_reduce, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    conv_3x3 = Conv2D(filters_3x3, (3, 3), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_3x3)

    conv_5x5 = Conv2D(filters_5x5_reduce, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    conv_5x5 = Conv2D(filters_5x5, (5, 5), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_5x5)

    pool_proj = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
    pool_proj = Conv2D(filters_pool_proj, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(pool_proj)

    output = concatenate([conv_1x1, conv_3x3, conv_5x5, pool_proj], axis=3, name=name)
    
    return output

In [ ]:
import keras

kernel_init = keras.initializers.glorot_uniform()
bias_init = keras.initializers.Constant(value=0.2)

In [ ]:
from keras.layers import concatenate
from keras.layers import Input

input_layer = Input(shape=(200, 200, 3))

x = Conv2D(64, (7, 7), padding='same', strides=(2, 2), activation='relu', name='conv_1_7x7/2', kernel_initializer=kernel_init, bias_initializer=bias_init)(input_layer)
x = MaxPooling2D((3, 3), padding='same', strides=(2, 2), name='max_pool_1_3x3/2')(x)
x = Conv2D(64, (1, 1), padding='same', strides=(1, 1), activation='relu', name='conv_2a_3x3/1')(x)
x = Conv2D(192, (3, 3), padding='same', strides=(1, 1), activation='relu', name='conv_2b_3x3/1')(x)
x = MaxPooling2D((3, 3), padding='same', strides=(2, 2), name='max_pool_2_3x3/2')(x)

x = inception_module(x,
                     filters_1x1=64,
                     filters_3x3_reduce=96,
                     filters_3x3=128,
                     filters_5x5_reduce=16,
                     filters_5x5=32,
                     filters_pool_proj=32,
                     name='inception_3a')

x = inception_module(x,
                     filters_1x1=128,
                     filters_3x3_reduce=128,
                     filters_3x3=192,
                     filters_5x5_reduce=32,
                     filters_5x5=96,
                     filters_pool_proj=64,
                     name='inception_3b')

x = MaxPooling2D((3, 3), padding='same', strides=(2, 2), name='max_pool_3_3x3/2')(x)

x = inception_module(x,
                     filters_1x1=192,
                     filters_3x3_reduce=96,
                     filters_3x3=208,
                     filters_5x5_reduce=16,
                     filters_5x5=48,
                     filters_pool_proj=64,
                     name='inception_4a')


x1 = AveragePooling2D((5, 5), strides=3)(x)
x1 = Conv2D(128, (1, 1), padding='same', activation='relu')(x1)
x1 = Flatten()(x1)
x1 = Dense(1024, activation='relu')(x1)
x1 = Dropout(0.7)(x1)
x1 = Dense(1, activation='sigmoid', name='auxilliary_output_1')(x1)

x = inception_module(x,
                     filters_1x1=160,
                     filters_3x3_reduce=112,
                     filters_3x3=224,
                     filters_5x5_reduce=24,
                     filters_5x5=64,
                     filters_pool_proj=64,
                     name='inception_4b')

x = inception_module(x,
                     filters_1x1=128,
                     filters_3x3_reduce=128,
                     filters_3x3=256,
                     filters_5x5_reduce=24,
                     filters_5x5=64,
                     filters_pool_proj=64,
                     name='inception_4c')

x = inception_module(x,
                     filters_1x1=112,
                     filters_3x3_reduce=144,
                     filters_3x3=288,
                     filters_5x5_reduce=32,
                     filters_5x5=64,
                     filters_pool_proj=64,
                     name='inception_4d')


x2 = AveragePooling2D((5, 5), strides=3)(x)
x2 = Conv2D(128, (1, 1), padding='same', activation='relu')(x2)
x2 = Flatten()(x2)
x2 = Dense(1024, activation='relu')(x2)
x2 = Dropout(0.7)(x2)
x2 = Dense(1, activation='sigmoid', name='auxilliary_output_2')(x2)

x = inception_module(x,
                     filters_1x1=256,
                     filters_3x3_reduce=160,
                     filters_3x3=320,
                     filters_5x5_reduce=32,
                     filters_5x5=128,
                     filters_pool_proj=128,
                     name='inception_4e')

x = MaxPooling2D((3, 3), padding='same', strides=(2, 2), name='max_pool_4_3x3/2')(x)

x = inception_module(x,
                     filters_1x1=256,
                     filters_3x3_reduce=160,
                     filters_3x3=320,
                     filters_5x5_reduce=32,
                     filters_5x5=128,
                     filters_pool_proj=128,
                     name='inception_5a')

x = inception_module(x,
                     filters_1x1=384,
                     filters_3x3_reduce=192,
                     filters_3x3=384,
                     filters_5x5_reduce=48,
                     filters_5x5=128,
                     filters_pool_proj=128,
                     name='inception_5b')

x = GlobalAveragePooling2D(name='avg_pool_5_3x3/1')(x)

x = Dropout(0.4)(x)

x = Dense(1, activation='sigmoid', name='output')(x)

In [ ]:
from keras.models import Model

model = Model(input_layer, [x, x1, x2], name='inception_v1')

In [ ]:
model.summary()

In [ ]:
%%time

from keras.optimizers import SGD
from keras.callbacks import LearningRateScheduler
import math

initial_lrate = 0.10

def decay(epoch, steps=100):
    initial_lrate = 0.01
    drop = 0.96
    epochs_drop = 8
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate

sgd = SGD(lr=initial_lrate, momentum=0.9, nesterov=False)

lr_sc = LearningRateScheduler(decay, verbose=1)

# model.compile(loss=['binary_crossentropy', 'binary_crossentropy', 'binary_crossentropy'], loss_weights=[1, 0.3, 0.3], optimizer="adam", metrics=['accuracy'])


In [ ]:
# model.load_weights('inception-model.h5')

In [ ]:
%%time
model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])
history = History()
model.fit(x_train, [y_train, y_train, y_train], validation_data = (x_test, [y_test, y_test, y_test]), epochs = 50, callbacks = [lr_sc, history], batch_size = 16)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# plot the training loss and accuracy
def plotResults(): 
    plt.figure()
    N = 50
    
    # TODO: plot the accuracy/loss variables over training time
    plt.plot(np.arange(0, N), history.history["output_accuracy"], label = "train_acc")
    plt.plot(np.arange(0, N), history.history["val_output_accuracy"], label = "val_acc")

    # make the graph understandable: 
    plt.title("Training Accuracy")
    plt.xlabel("Epoch #")
    plt.ylabel("Accuracy")
    plt.yscale('log')
    plt.legend(loc="lower right")
    plt.show()
    
    plt.figure()
    plt.plot(np.arange(0, N), history.history["output_loss"], label = "train_loss")
    plt.plot(np.arange(0, N), history.history["val_output_loss"], label = "val_loss")
    
     # make the graph understandable: 
    plt.title("Training Loss")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss")
    plt.yscale('log')
    plt.legend(loc="upper right")
    plt.show()

In [ ]:
model.save_weights('inception-model.h5')
plotResults()

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt

def evaluate_on_training_set(y_test, y_pred):
    
    # Calculate AUC
    print("AUC  is:  ",  roc_auc_score(y_test,  y_pred))
    
    # print out recall and precision
    print(classification_report(y_test, y_pred))
    
    # print out confusion matrix
    print("Confusion Matrix: \n", confusion_matrix(y_test, y_pred))
    
    # # calculate points for ROC curve
    fpr, tpr, thresholds = roc_curve(y_test, y_pred)
    
    # Plot ROC curve
    plt.plot(fpr,  tpr,  label='ROC  curve  (area  =  %0.3f)'  %  roc_auc_score(y_test,  y_pred)) 
    plt.plot([0,  1],  [0,  1],  'k--') #  random  predictions  curve
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('False  Positive  Rate  or  (1  -  Specifity)') 
    plt.ylabel('True  Positive  Rate  or  (Sensitivity)') 
    plt.title('Receiver  Operating  Characteristic')


In [ ]:
score = model.evaluate(eval_x, [eval_y, eval_y, eval_y], verbose = 1)

In [ ]:
import tensorflow as tf
output = model.predict(eval_x, verbose = 1)
pred_y = np.array(output).flatten()

In [ ]:
print(eval_y)
print(pred_y)

In [ ]:
print(f"Loss: {score[0]}\nAccuracy: {score[1]}")
evaluate_on_training_set(eval_y, pred_y)